In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
162,AFG,Asia,Afghanistan,2020-06-20,27878.0,346.0,548.0,2.0,716.136,8.888,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
266,ALB,Europe,Albania,2020-06-20,1838.0,50.0,42.0,3.0,638.682,17.374,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
434,DZA,Africa,Algeria,2020-06-20,11504.0,119.0,825.0,14.0,262.343,2.714,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-19,France,159106.0,0.0,29634.0,0.0,0.0,0.0
0,2020-06-19,China,84567.0,0.0,4650.0,0.0,0.0,0.0
0,2020-06-19,Italy,238610.0,0.0,34579.0,0.0,0.0,0.0
0,2020-06-19,Spain,245676.0,0.0,29068.0,0.0,0.0,0.0
0,2020-06-19,United States,2218413.0,0.0,119326.0,0.0,0.0,0.0
0,2020-06-19,World,8631288.0,0.0,461396.0,0.0,0.0,0.0
0,2020-06-19,United Kingdom,301971.0,0.0,42516.0,0.0,0.0,0.0
0,2020-06-19,Germany,188394.0,0.0,8890.0,0.0,0.0,0.0
0,2020-06-19,Iran,200495.0,0.0,9387.0,0.0,0.0,0.0
0,2020-06-19,Turkey,185361.0,0.0,4900.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-20,France,159106.0,159452.0,29634.0,29617.0,-346.0,17.0
0,2020-06-20,China,84567.0,84524.0,4650.0,4638.0,43.0,12.0
0,2020-06-20,Italy,238610.0,238011.0,34579.0,34561.0,599.0,18.0
0,2020-06-19,Spain,245676.0,245575.0,29068.0,30233.0,101.0,-1165.0
0,2020-06-20,United States,2218413.0,2220961.0,119326.0,119112.0,-2548.0,214.0
0,2020-06-20,World,8631288.0,8633778.0,461396.0,462076.0,-2490.0,-680.0
0,2020-06-20,United Kingdom,301971.0,301815.0,42516.0,42461.0,156.0,55.0
0,2020-06-20,Germany,188394.0,189135.0,8890.0,8883.0,-741.0,7.0
0,2020-06-20,Iran,200495.0,200262.0,9387.0,9392.0,233.0,-5.0
0,2020-06-20,Turkey,185361.0,185245.0,4900.0,4905.0,116.0,-5.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")